In [1]:
import time
import json

import numpy as np
import pandas as pd

In [2]:
def calculate_robustness_score_for_dataset(
        df,
        dataset_type,
        postprocessors,
        activations,
        optimizers,
        metric):
    stats_for_pp = {}
    for pp in postprocessors:
        total_conf = 0
        distribution_stats_per_a_o = {}
        sub_df = df[(df['Dataset'] == dataset_type) & (df['PostProcessor'] == pp)]
        for a in activations:
            for o in optimizers:
                key = a+"_"+o

                sub_sub_df = sub_df[(sub_df['Activation'] == a) & (sub_df['Optimizer'] == o)][metric]
                conf =  1/np.sqrt(sub_sub_df.var())
                distribution_stats_per_a_o[key] = {'mean': sub_sub_df.mean(), 
                                                      'var': sub_sub_df.var(), 
                                                      'conf': conf}    
                total_conf += conf

        for key, stats in distribution_stats_per_a_o.items():
            stats['weight'] = stats['conf']/total_conf
        stats_for_pp[pp] = distribution_stats_per_a_o

    final_mean_var_pp = {}

    for pp, a_o_stats in stats_for_pp.items():
        weighted_mean = 0
        for key, per_a_o_stats in a_o_stats.items():
            weighted_mean += (per_a_o_stats['mean'] * per_a_o_stats['weight'])
        final_mean_var_pp[pp] = { 'weighted_mean': weighted_mean }

        weighted_var = 0
        for key, per_a_o_stats in a_o_stats.items():
            weighted_var += per_a_o_stats['weight']*(per_a_o_stats['var'] + (weighted_mean - per_a_o_stats['mean'])**2)
        final_mean_var_pp[pp]['weighted_var'] = weighted_var

    for pp, weighted_stats in final_mean_var_pp.items():
        print(pp, np.sqrt(weighted_stats['weighted_var'])/weighted_stats['weighted_mean'])

In [3]:
def calculate_marginal_for_activations(
        df,
        dataset_type,
        postprocessors,
        activations,
        optimizers,
        metric):
    stats_for_pp = {}
    for pp in postprocessors:
        total_conf = 0
        distribution_stats_per_a_o = {}
        sub_df = df[(df['Dataset'] == dataset_type) & (df['PostProcessor'] == pp)]
        for a in activations:
            key = a
            sub_sub_df = sub_df[(sub_df['Activation'] == a)][metric]
            conf =  1/np.sqrt(sub_sub_df.var())
            distribution_stats_per_a_o[key] = {'mean': sub_sub_df.mean(), 
                                                  'var': sub_sub_df.var(), 
                                                  'conf': conf}    
            total_conf += conf

        for key, stats in distribution_stats_per_a_o.items():
            stats['weight'] = stats['conf']/total_conf
        stats_for_pp[pp] = distribution_stats_per_a_o
    
    print(stats_for_pp)
    final_mean_var_pp = {}

    for pp, a_o_stats in stats_for_pp.items():
        weighted_mean = 0
        for key, per_a_o_stats in a_o_stats.items():
            weighted_mean += (per_a_o_stats['mean'] * per_a_o_stats['weight'])
        final_mean_var_pp[pp] = { 'weighted_mean': weighted_mean }

        weighted_var = 0
        for key, per_a_o_stats in a_o_stats.items():
            weighted_var += per_a_o_stats['weight']*(per_a_o_stats['var'] + (weighted_mean - per_a_o_stats['mean'])**2)
        final_mean_var_pp[pp]['weighted_var'] = weighted_var

    for pp, weighted_stats in final_mean_var_pp.items():
        print(pp, np.sqrt(weighted_stats['weighted_var'])/weighted_stats['weighted_mean'])

In [4]:
def calculate_marginal_for_optimizers(
        df,
        dataset_type,
        postprocessors,
        activations,
        optimizers,
        metric):
    stats_for_pp = {}
    for pp in postprocessors:
        total_conf = 0
        distribution_stats_per_a_o = {}
        sub_df = df[(df['Dataset'] == dataset_type) & (df['PostProcessor'] == pp)]
        for o in optimizers:
            key = o
            sub_sub_df = sub_df[(sub_df['Optimizer'] == o)][metric]
            conf =  1/np.sqrt(sub_sub_df.var())
            distribution_stats_per_a_o[key] = {'mean': sub_sub_df.mean(), 
                                                  'var': sub_sub_df.var(), 
                                                  'conf': conf}    
            total_conf += conf

        for key, stats in distribution_stats_per_a_o.items():
            stats['weight'] = stats['conf']/total_conf
        stats_for_pp[pp] = distribution_stats_per_a_o
    
    print(stats_for_pp)
    final_mean_var_pp = {}

    for pp, a_o_stats in stats_for_pp.items():
        weighted_mean = 0
        for key, per_a_o_stats in a_o_stats.items():
            weighted_mean += (per_a_o_stats['mean'] * per_a_o_stats['weight'])
        final_mean_var_pp[pp] = { 'weighted_mean': weighted_mean }

        weighted_var = 0
        for key, per_a_o_stats in a_o_stats.items():
            weighted_var += per_a_o_stats['weight']*(per_a_o_stats['var'] + (weighted_mean - per_a_o_stats['mean'])**2)
        final_mean_var_pp[pp]['weighted_var'] = weighted_var

    for pp, weighted_stats in final_mean_var_pp.items():
        print(pp, np.sqrt(weighted_stats['weighted_var'])/weighted_stats['weighted_mean'])

### Robustness Score For MNIST

In [5]:
df = pd.read_csv('oodn_odin_mcd_results_v1.csv')
dataset_type = 'mnist'
postprocessors = ['mcd', 'odin']
activations = ['relu', 'softplus']
optimizers = ['SGD', 'Adam']
metric = 'AUROC'

calculate_robustness_score_for_dataset(
        df,
        dataset_type,
        postprocessors,
        activations,
        optimizers,
        metric)

mcd 0.019863695545259616
odin 0.007995262031607268


In [6]:
df = pd.read_csv('oodn_odin_mcd_results_v1.csv')
dataset_type = 'mnist'
postprocessors = ['mcd', 'odin']
activations = ['relu', 'softplus']
optimizers = ['SGD', 'Adam']
metric = 'AUROC'

calculate_marginal_for_activations(
        df,
        dataset_type,
        postprocessors,
        activations,
        optimizers,
        metric)

{'mcd': {'relu': {'mean': 98.61666666666667, 'var': 0.34846666666667114, 'conf': 1.6940233002084395, 'weight': 0.8714385215699936}, 'softplus': {'mean': 95.06, 'var': 16.010799999999993, 'conf': 0.2499156676908309, 'weight': 0.12856147843000637}}, 'odin': {'relu': {'mean': 99.49166666666667, 'var': 0.05113666666666559, 'conf': 4.422153284483592, 'weight': 0.9229650098385574}, 'softplus': {'mean': 97.19, 'var': 7.34052, 'conf': 0.3690936613319411, 'weight': 0.07703499016144251}}}
mcd 0.019804813576411687
odin 0.01001523450658029


In [10]:
df = pd.read_csv('oodn_odin_mcd_results_v1.csv')
dataset_type = 'mnist'
postprocessors = ['mcd', 'odin']
activations = ['relu', 'softplus']
optimizers = ['SGD', 'Adam']
metric = 'AUROC'

calculate_marginal_for_optimizers(
        df,
        dataset_type,
        postprocessors,
        activations,
        optimizers,
        metric)

{'mcd': {'SGD': {'mean': 98.17333333333333, 'var': 1.050666666666673, 'conf': 0.9755904108519041, 'weight': 0.8080575290465987}, 'Adam': {'mean': 95.50333333333333, 'var': 18.621186666666695, 'conf': 0.2317375030442415, 'weight': 0.19194247095340128}}, 'odin': {'SGD': {'mean': 99.265, 'var': 0.11954999999999902, 'conf': 2.8921792755534153, 'weight': 0.8934218275207201}, 'Adam': {'mean': 97.41666666666667, 'var': 8.400906666666668, 'conf': 0.3450141603617603, 'weight': 0.1065781724792799}}}
mcd 0.02407678915650231
odin 0.011629947691945634


### Robustness Score For CIFAR

In [7]:
df = pd.read_csv('oodn_odin_mcd_results_v1.csv')
dataset_type = 'cifar'
postprocessors = ['mcd', 'odin']
activations = ['relu', 'softplus']
optimizers = ['SGD', 'Adam']
metric = 'AUROC'

calculate_robustness_score_for_dataset(
        df,
        dataset_type,
        postprocessors,
        activations,
        optimizers,
        metric)

mcd 0.04288593604590558
odin 0.030402886587164866


In [8]:
df = pd.read_csv('oodn_odin_mcd_results_v1.csv')
dataset_type = 'cifar'
postprocessors = ['mcd', 'odin']
activations = ['relu', 'softplus']
optimizers = ['SGD', 'Adam']
metric = 'AUROC'

calculate_marginal_for_activations(
        df,
        dataset_type,
        postprocessors,
        activations,
        optimizers,
        metric)

{'mcd': {'relu': {'mean': 67.925, 'var': 12.378909999999992, 'conf': 0.28422272390517755, 'weight': 0.2865565239331451}, 'softplus': {'mean': 65.31333333333333, 'var': 1.9970266666666696, 'conf': 0.7076329840161257, 'weight': 0.7134434760668549}}, 'odin': {'relu': {'mean': 79.33166666666666, 'var': 4.5995366666666735, 'conf': 0.46627588744642, 'weight': 0.665991264045911}, 'softplus': {'mean': 77.40333333333334, 'var': 18.286706666666685, 'conf': 0.23384724121713654, 'weight': 0.33400873595408903}}}
mcd 0.038194411089270956
odin 0.040184407953758355


In [9]:
df = pd.read_csv('oodn_odin_mcd_results_v1.csv')
dataset_type = 'cifar'
postprocessors = ['mcd', 'odin']
activations = ['relu', 'softplus']
optimizers = ['SGD', 'Adam']
metric = 'AUROC'

calculate_marginal_for_optimizers(
        df,
        dataset_type,
        postprocessors,
        activations,
        optimizers,
        metric)

{'mcd': {'SGD': {'mean': 67.40666666666667, 'var': 12.67694666666667, 'conf': 0.2808617958288894, 'weight': 0.3681352195529563}, 'Adam': {'mean': 65.83166666666666, 'var': 4.303096666666661, 'conf': 0.4820692711033997, 'weight': 0.6318647804470438}}, 'odin': {'SGD': {'mean': 76.485, 'var': 13.145270000000014, 'conf': 0.2758133254810892, 'weight': 0.3393039239929954}, 'Adam': {'mean': 80.25, 'var': 3.466920000000003, 'conf': 0.537066532303235, 'weight': 0.6606960760070046}}}
mcd 0.04249037015295456
odin 0.03989950723904008


In [23]:
df[(df['Dataset']=='cifar') & (df['Optimizer'] == 'Adam') & (df['PostProcessor'] == 'odin')]['AUROC'].mean()

80.25